In [2]:
# make all necessary imports here
import pandas as pd

In [3]:
SUM_INTERVALS = [(-7,0), (-14,0), (-30,0)]

## Generate some random data

This function helps you generate some random test data. Call this for your testing/developement purposes. Please don't call this in your final submission as it will slow down our autmatic tests.

In [4]:
from challenge import generate_data

In [5]:
generate_data??

In [6]:
generate_data()

100%|██████████| 32/32 [00:07<00:00,  4.05it/s]


# Challenge
The `SUM_INTERVAL` variable contains relative time intervals in days.
So the first means 7 days back until today (asof time writing this 2016-02-01).

The generated files contain 1000 distinct users which made visits over the timespan of one month.
Each visit has a value for `feature a`, as awell as `feature_b` assigned to it.

**For each user calculate the sum of it's respective features for each time interval.
The final output should be a dataframe or numpy matrix containing one row per user,
an id column an the feature columns (N_users, 1 + N_intervals*N_features)**

### Note
The feature columns are expected to be in descending order (in terms of days included, so
30 days feature A as the first column and 30 days feature B is second and so on). 
The resulting matrix should be ordered by the user ids.

When the relative interval `(-7, 0)` is given with the date `2017-02-01` the semantics 
are to include features values within following time interval: 
`[2016-01-25 00:00:00; 2016-02-01 00:00:00)`

*You are encouraged to use the pandas library for this task but it is not required.*

In [5]:
import dask.dataframe as dd
import numpy as np

def bin_sum_features(csv_glob="data/shard-*.csv.gz", today=pd.Timestamp('2016-02-01')):
    """Calculate sum of feature for each time interval using Dask"""
    
    dfs = []
    
    if not SUM_INTERVALS:
        return None
    
    df = dd.read_csv(csv_glob, parse_dates=['timestamp'], compression='gzip', blocksize=None)

    for interval in SUM_INTERVALS:

        days_before = interval[0]
        days_after = interval[1]
        start_day = today + pd.Timedelta(f"{days_before} days")
        end_day = today + pd.Timedelta(f"{days_after} days")

        if start_day > end_day:
            raise ValueError("Wrong day interval")

        mask = (df['timestamp'] >= start_day) & (df['timestamp'] <= end_day)

        interval_data = df.mask(mask, other=None)
        aggregate = interval_data.groupby('id').sum()
        
        days = abs(days_before)
        aggregate = aggregate.rename(columns={"feature_a": f"feature_a_{days}", "feature_b": f"feature_b_{days}"})
        
        dfs.append(aggregate)
        
    df_aggregate = dfs[0]
    
    for frame in dfs[1:]:
        df_aggregate = df_aggregate.merge(frame, how='outer')
        
    return df_aggregate.reset_index()


bin_sum_features().compute().head()



,id,feature_a_7,feature_b_7,feature_a_14,feature_b_14,feature_a_30,feature_b_30
0,0092505e-4843-40ba-81f7-f8c6791d33c5,3746,34.723856,2483,25.919248,257.0,3.741570
1,017066dd-d682-46fa-b291-98ee96afc70c,4910,44.706509,3678,33.896826,205.0,1.837862
2,019136f7-c04a-4e5d-85c8-d861dce5780a,3892,33.182412,2809,21.631453,120.0,0.974517
3,01add7e9-5c50-4952-872e-4b4b863d8290,3692,40.812955,3019,29.810611,38.0,0.808364
4,01b01b82-31b0-4d9b-8ce3-2177f83d7eef,4211,45.030768,3071,34.234082,362.0,4.047624


## Result shape example
below you see how the results could look like with pandas

In [7]:
pd.DataFrame([['1aa9204b-5956-41a3-96b6-58cbf6bc147e',1,2,3,4,5,6]], 
             columns=['id', 'feature_a_7', 'feature_a_14', 'feature_a_30', 
                      'feature_b_7', 'feature_b_14', 'feature_b_30'])

,id,feature_a_7,feature_a_14,feature_a_30,feature_b_7,feature_b_14,feature_b_30
0,1aa9204b-5956-41a3-96b6-58cbf6bc147e,1,2,3,4,5,6
